# Projeto 1 - Ciência dos Dados

Nome: Carlos Eduardo Porciuncula Yamada

Nome: Pedro Henrique de Sousa da Silva

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Obtenção de tweets

## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [ ]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@projetocdados1***

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @projetocdados1

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas para construção da base de dados:

### 1. Escolha de um produto e coleta das mensagens


In [4]:
#Produto escolhido:
produto = 'Apple'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    
    try:
        #Tenta acessar atributo 'retweeted_status'
        # Se mensagem não possuir este atributo dá erro
        # e vai para a clausula "except"
        msg.retweeted_status.full_text
    except AttributeError:  
        #Entra aqui sempre que tweet não for retweet
        msgs.append(msg.full_text.lower())

    
    i += 1
    
    temporario=list(set(list(msgs)))
    if len(temporario) >= n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)
len(msgs)

In [ ]:
msgs=list(set(list(msgs)))

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

**Importante: Caso classifique um percentual pequeno de tweets relevantes ou de não relevantes, deve voltar a este notebook e coletar mais tweets diferentes do produto escolhido pelo grupo.**

In [7]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Paula Yamada\OneDrive\Área de Trabalho\2021.2\cdados\cdados-projeto-1


In [8]:
dft = pd.read_excel('Apple.xlsx', sheet_name='Treinamento')

In [10]:
dfc = pd.read_excel('Apple.xlsx', sheet_name='Teste')

In [22]:
# ABA 'Treinamento' DA PLANILHA 'Apple.xlsx'
tbin_r = dft.isin([1])

tfr = tbin_r['Classificação'] == True
tfi = tbin_r['Classificação'] == False

tdfr = dft.loc[tfr, :]
tdfi = dft.loc[tfi, :]

In [23]:
# ABA 'Teste' DA PLANILHA 'Apple.xlsx'
cbin_r = dfc.isin([1])
cfr = cbin_r['Classificação'] == True
cfi = cbin_r['Classificação'] == False

cdfr = dfc.loc[cfr, :]
cdfi = dfc.loc[cfi, :]